In [105]:
from pathlib import Path
import pandas as pd


csv = Path.cwd()/'2022-01-14-17-00_influxdb_data.csv'
data = pd.read_csv(csv, header=2, skiprows=range(2,4))

In [106]:
data.head()

,Unnamed: 0,result,table,_start,_stop,_time,_value,_field,_measurement,driver
0,NaN,NaN,0,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-01-01T16:00:00Z,51.6,consumed,charge,Driver 1
1,NaN,NaN,0,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-01-05T12:00:00Z,55.5,consumed,charge,Driver 1
2,NaN,NaN,0,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-01-09T16:00:00Z,45.8,consumed,charge,Driver 1
3,NaN,NaN,0,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-01-14T10:00:00Z,54.9,consumed,charge,Driver 1
4,NaN,NaN,0,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-01-20T17:00:00Z,54.0,consumed,charge,Driver 1


In [107]:
negative_values = data[data['_value'] < 0]
negative_values.head()

,Unnamed: 0,result,table,_start,_stop,_time,_value,_field,_measurement,driver
223,NaN,NaN,2,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-05-30T13:00:00Z,-1.03,consumed,charge,Driver 5
226,NaN,NaN,2,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-06-07T18:00:00Z,-20.54,consumed,charge,Driver 5
256,NaN,NaN,2,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-07-13T16:00:00Z,-0.21,consumed,charge,Driver 5
262,NaN,NaN,2,2020-12-31T23:00:00Z,2021-12-31T06:57:17.258Z,2021-08-01T16:00:00Z,-0.21,consumed,charge,Driver 5


## I have observed negative values in the consumption field. Does this mean that the charging can be bi-directional? Perhaps some kind of refunds? In this case, negative values are accepted. 
## Otherwise, these entries indicate errors and they should be removed.
## Here bi-directional charging is assumed.

In [118]:
# In the non bi-directional charging case, this step would remove negative values
# if data['_value'].dtype == 'O':
#     data['_value'] = pd.to_numeric(data['_value'], errors='coerce')
# data=data[data['_value']>0]

In [109]:
# Convert '_time' to datetime and extract 'month' and 'year'
if data['_time'].dtype == 'O':
    data['_time'] = pd.to_datetime(data['_time'])

data['month'] = data['_time'].apply(lambda x: x.month)
data['year'] = data['_time'].apply(lambda x: x.year)

## Total number of charging sessions per driver, per month and for the whole year

In [110]:
# Total number of charging sessions per driver per month
sessions_per_driver = data.groupby(['driver', 'month']).size().reset_index(name='Sessions_per_month')
sessions_per_driver

,driver,month,Sessions_per_month
0,Driver 1,1,8
1,Driver 1,2,8
2,Driver 1,3,14
3,Driver 1,4,14
4,Driver 1,5,10
5,Driver 1,6,17
6,Driver 1,7,14
7,Driver 1,8,5
8,Driver 1,9,4
9,Driver 1,10,10


In [111]:
# Total number of charging sessions per driver per year
sessions_per_driver_per_year = data.groupby(['year','driver']).size().reset_index(name='Sessions_per_year')
sessions_per_driver_per_year

,year,driver,Sessions_per_year
0,2021,Driver 1,127
1,2021,Driver 2,74
2,2021,Driver 5,76


## Total and average number of charging sessions for the total fleet (of 3 drivers), per month and for the whole year

In [112]:
# Total and average number of charging sessions for the fleet per month
sessions_fleet_yearly = data.groupby(['year','month']).size().reset_index(name='Total_sessions_fleet')
sessions_fleet_yearly['Average sessions'] = sessions_fleet_yearly['Total_sessions_fleet'] / len(data['driver'].unique())
print(sessions_fleet_yearly)

# For the whole year
whole_year = sessions_fleet_yearly['Total_sessions_fleet'].sum()
print(f"Total number of charging sessions for the whole fleet in a year: {whole_year} and average: {whole_year.mean()}")

    year  month  Total_sessions_fleet  Average sessions
0   2021      1                    17          5.666667
1   2021      2                    16          5.333333
2   2021      3                    20          6.666667
3   2021      4                    32         10.666667
4   2021      5                    23          7.666667
5   2021      6                    38         12.666667
6   2021      7                    36         12.000000
7   2021      8                    10          3.333333
8   2021      9                     7          2.333333
9   2021     10                    18          6.000000
10  2021     11                    29          9.666667
11  2021     12                    31         10.333333
Total number of charging sessions for the whole fleet in a year: 277 and average: 277.0


## Total and average kWh per driver, per month and for the whole year

In [113]:
# Total and average kWh per driver, per month
consumption_per_driver = data.groupby(['driver', 'month'])['_value'].sum().reset_index(name='Total_kWh_per_month')

# Average consumption per driver per month
consumption_per_driver['Average_kWh_per_month'] = consumption_per_driver['Total_kWh_per_month'] / sessions_per_driver['Sessions_per_month']
consumption_per_driver

,driver,month,Total_kWh_per_month,Average_kWh_per_month
0,Driver 1,1,402.700,50.337500
1,Driver 1,2,327.600,40.950000
2,Driver 1,3,430.880,30.777143
3,Driver 1,4,527.000,37.642857
4,Driver 1,5,423.360,42.336000
5,Driver 1,6,483.454,28.438471
6,Driver 1,7,492.588,35.184857
7,Driver 1,8,197.010,39.402000
8,Driver 1,9,155.490,38.872500
9,Driver 1,10,366.456,36.645600


In [114]:
# Total consumption per driver per year
consumption_per_driver_year = data.groupby(['driver', 'year'])['_value'].sum().reset_index(name='Total_kWh_per_year')

# Average consumption per station visit per year
consumption_per_driver_year['Average_kWh_per_year'] = consumption_per_driver_year['Total_kWh_per_year']/ sessions_per_driver_per_year['Sessions_per_year']
consumption_per_driver_year

,driver,year,Total_kWh_per_year,Average_kWh_per_year
0,Driver 1,2021,4642.904,36.558299
1,Driver 2,2021,1614.280,21.814595
2,Driver 5,2021,1756.989,23.118276


## Total and average kWh for the total fleet (of 3 drivers), per month and for the whole year

In [115]:
# Total kWh for the total fleet (of 3 drivers), per month
consumption_fleet_month = data.groupby(['month'])['_value'].sum().reset_index(name='Total_kWh_fleet_per_month')

# Average kWh for the total fleet (of 3 driver), per month
consumption_fleet_month['Average kWh'] = consumption_fleet_month['Total_kWh_fleet_per_month']/sessions_per_driver['Sessions_per_month']
consumption_fleet_month

,month,Total_kWh_fleet_per_month,Average kWh
0,1,715.480,89.435000
1,2,533.900,66.737500
2,3,590.490,42.177857
3,4,1011.160,72.225714
4,5,759.212,75.921200
5,6,814.483,47.910765
6,7,746.018,53.287000
7,8,244.400,48.880000
8,9,215.720,53.930000
9,10,591.666,59.166600


In [116]:
# Total kWh for the total fleet (of 3 drivers), per year
consumption_fleet_year = data.groupby(['year'])['_value'].sum().reset_index(name='Total_kWh_fleet_per_year')

# Average kWh for the total fleet (of 3 drivers), per year
consumption_fleet_year['Average kWh'] = consumption_fleet_year['Total_kWh_fleet_per_year']/len(data['driver'].unique())
consumption_fleet_year

,year,Total_kWh_fleet_per_year,Average kWh
0,2021,8014.173,2671.391


In [102]:
# Merging dataframes per category
merged_per_driver_year = pd.merge(sessions_per_driver_per_year, consumption_per_driver_year, on=['driver','year'])
merged_per_driver = pd.merge(sessions_per_driver, consumption_per_driver, on=['driver', 'month'])
merged_fleet = pd.merge(consumption_fleet_month, sessions_fleet_yearly, on='month')

In [103]:
# Exporting CSV files
merged_per_driver.to_csv('Stats_per_driver.csv', index=False)
merged_per_driver_year.to_csv('Stats_per_year.csv', index=False)
merged_fleet.to_csv('Stats_for_fleet.csv', index=False)
consumption_fleet_year.to_csv('consumption_fleet_year.csv', index=False)